In [19]:
# Instalar dependencias necesarias
!pip install mlflow dagshub --quiet


In [29]:
# 1. Configuración del entorno e importaciones
# --------------------------------------------
import os
import pandas as pd
import numpy as np
from getpass import getpass
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import mlflow
import mlflow.sklearn

In [ ]:
# -------------------- CONFIGURACIÓN MLFLOW --------------------

# Finalizar cualquier sesión de MLflow activa
try:
    mlflow.end_run()
    print("Sesión anterior de MLflow finalizada")
except:
    pass

# Configurar MLflow
try:
    print("Configurando MLflow...")
    os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow'
    os.environ['MLFLOW_TRACKING_USERNAME'] = 'AngelBReal'

    # Solo solicita el token si no está configurado
    if 'MLFLOW_TRACKING_PASSWORD' not in os.environ:
        os.environ['MLFLOW_TRACKING_PASSWORD'] = getpass('Token DagsHub: ')

    # Configurar URI
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
    print(f"URI de tracking: {mlflow.get_tracking_uri()}")
except Exception as e:
    print(f"Error en configuración remota: {e}")
    print("Usando MLflow en modo local")
    mlflow.set_tracking_uri(None)

# -------------------- CARGA Y PREPARACIÓN DE DATOS --------------------

# Cargar dataset
print("\nCargando dataset...")
df = pd.read_csv("/content/classical_models.csv")
print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")

# Limpiar valores faltantes
df_clean = df.dropna(subset=['label'])
print(f"Dataset después de limpiar: {df_clean.shape[0]} filas")

# Separar características y variable objetivo
X = df_clean.drop(columns=["label"])
y = df_clean["label"]

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

print(f"Conjunto de entrenamiento: {X_train.shape}")
print(f"Conjunto de prueba: {X_test.shape}")

# -------------------- ENTRENAMIENTO Y EVALUACIÓN DEL MODELO --------------------

# Establece el nombre del experimento
experiment_name = "Regresión Noticias Falsas"
mlflow.set_experiment(experiment_name)
print(f"Experimento configurado: {experiment_name}")

# Definición de la rejilla de hiperparámetros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 5]
}

# Inicia la ejecución principal
with mlflow.start_run(run_name="GridSearchCV con ejecuciones anidadas") as parent_run:
    print(f"\nEjecución principal iniciada: {parent_run.info.run_id}")

    # Registra el dataset
    dataset = mlflow.data.from_pandas(df_clean, name="classical_models", targets="label")
    mlflow.log_input(dataset)

    # Establece etiquetas para la ejecución
    mlflow.set_tags({
        "modelo": "RandomForestRegressor",
        "preprocesamiento": "TF-IDF",
        "experimentador": "AngelBReal"
    })

    # Variables para rastrear el mejor modelo
    best_model = None
    best_score = float('-inf')
    best_params = None

    # Número total de combinaciones
    total_combinations = len(list(ParameterGrid(param_grid)))
    print(f"Total de combinaciones a probar: {total_combinations}")

    # Iteración sobre cada combinación de hiperparámetros
    for i, params in enumerate(ParameterGrid(param_grid)):
        # Inicialización del modelo con los parámetros actuales
        model = RandomForestRegressor(random_state=42, **params)

        print(f"\nProbando combinación {i+1}/{total_combinations}:")
        print(f"Parámetros: {params}")

        # Inicia una ejecución anidada para esta combinación de hiperparámetros
        with mlflow.start_run(run_name=f"Modelo con params: {params}", nested=True) as child_run:
            try:
                print(f"Ejecución anidada iniciada: {child_run.info.run_id}")

                # Entrenamiento del modelo
                print("Entrenando modelo...")
                model.fit(X_train, y_train)

                # Predicciones en el conjunto de prueba
                predictions = model.predict(X_test)

                # Cálculo de métricas
                mse = mean_squared_error(y_test, predictions)
                mae = mean_absolute_error(y_test, predictions)
                r2 = r2_score(y_test, predictions)

                # Mostrar métricas
                print(f"MSE: {mse:.6f}")
                print(f"MAE: {mae:.6f}")
                print(f"R²: {r2:.6f}")

                # Registro de parámetros y métricas en MLflow
                mlflow.log_params(params)
                mlflow.log_metric("mse", mse)
                mlflow.log_metric("mae", mae)
                mlflow.log_metric("r2_score", r2)

                # Crear y guardar visualización
                plt.figure(figsize=(10, 4))

                # Predicciones vs reales
                plt.subplot(1, 2, 1)
                plt.scatter(y_test, predictions, alpha=0.5)
                plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
                plt.xlabel('Valores reales')
                plt.ylabel('Predicciones')
                plt.title('Predicciones vs Valores reales')

                # Distribución de residuos
                plt.subplot(1, 2, 2)
                residuos = y_test - predictions
                sns.histplot(residuos, kde=True)
                plt.xlabel('Error')
                plt.title('Distribución de errores')

                plt.tight_layout()
                plt.savefig(f'model_evaluation_{i+1}.png')

                # Registrar visualización en MLflow
                mlflow.log_artifact(f'model_evaluation_{i+1}.png')

                # Registro del modelo
                mlflow.sklearn.log_model(model, "modelo_entrenado")

                # Actualizar el mejor modelo si es necesario
                if r2 > best_score:
                    best_score = r2
                    best_model = model
                    best_params = params
                    print("¡Nuevo mejor modelo encontrado!")

            except Exception as e:
                print(f"Error en ejecución anidada: {e}")

    # Registrar información del mejor modelo en la ejecución principal
    if best_model is not None:
        print("\n=== MEJOR MODELO ENCONTRADO ===")
        print(f"Parámetros: {best_params}")
        print(f"R²: {best_score:.6f}")

        # Registrar mejor modelo y parámetros en la ejecución principal
        mlflow.log_param("best_params", best_params)
        mlflow.log_metric("best_r2_score", best_score)

        # Guardar el mejor modelo con un nombre especial
        with open('best_model_params.txt', 'w') as f:
            f.write(f"Mejores parámetros: {best_params}\n")
            f.write(f"R² score: {best_score:.6f}\n")

        mlflow.log_artifact('best_model_params.txt')
    else:
        print("No se encontró un modelo válido")

print("\nProceso completado.")

Sesión anterior de MLflow finalizada
Configurando MLflow...
URI de tracking: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow

Cargando dataset...


<ipython-input-41-0d7bf3f5411c>:32: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,2

Dataset cargado: 3878 filas, 861 columnas
Dataset después de limpiar: 3063 filas
Conjunto de entrenamiento: (2450, 860)
Conjunto de prueba: (613, 860)


2025/05/05 06:28:21 INFO mlflow.tracking.fluent: Experiment with name 'Regresión Noticias Falsas' does not exist. Creating a new experiment.


Experimento configurado: Regresión Noticias Falsas

Ejecución principal iniciada: b13fe87aa6df4c27815d45fb20adff80
Total de combinaciones a probar: 12

Probando combinación 1/12:
Parámetros: {'max_depth': 5, 'min_samples_split': 2, 'n_estimators': 100}
Ejecución anidada iniciada: cafed377aac14db3a4e7063b4f4e5fce
Entrenando modelo...
MSE: 0.160283
MAE: 0.320930
R²: 0.265067


...

¡Nuevo mejor modelo encontrado!
🏃 View run Modelo con params: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100} at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4/runs/ec7d8c01da3746f2a25a605f3b246e0f
🧪 View experiment at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4

Probando combinación 8/12:
Parámetros: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}
Ejecución anidada iniciada: 6987f378e41949079540549c28ba4395
Entrenando modelo...
MSE: 0.152522
MAE: 0.296789
R²: 0.300653


2025/05/05 06:32:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Modelo con params: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200} at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4/runs/6987f378e41949079540549c28ba4395
🧪 View experiment at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4

Probando combinación 9/12:
Parámetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}
Ejecución anidada iniciada: 20c10797ffca40a6b895f553050a2e21
Entrenando modelo...
MSE: 0.146625
MAE: 0.284013
R²: 0.327690


2025/05/05 06:33:44 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


¡Nuevo mejor modelo encontrado!
🏃 View run Modelo con params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100} at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4/runs/20c10797ffca40a6b895f553050a2e21
🧪 View experiment at: https://dagshub.com/AngelBReal/DeAMentis-AAA.mlflow/#/experiments/4

Probando combinación 10/12:
Parámetros: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Ejecución anidada iniciada: 30133cbbde7b4ebba10ea82b10483382
Entrenando modelo...
